# Querying
Record times to execute queries in different databases of variable sizes

### Libraries

In [1]:
import pandas as pd
import mysql.connector
import os
import time
import pymongo
import datetime
import redis
from redisgraph import Graph
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from insert import ins

In [3]:
%load_ext dotenv
%dotenv

### Some Constants

In [4]:
MYSQL = "MySQL"
MONGO_DB = 'MongoDB'
REDIS = 'Redis'

In [5]:
HUNDRED = 'hundred'
FIVE_HUNDRED = 'five_hundred'
THOUSAND = 'thousand'
TEN_THOUSAND = 'ten_thousand'
LAKH = 'lakh'
FIVE_MILLION = 'five_mil'

### Query

In [6]:
def sql_query(query, cursor):
    begin = time.time()
    cursor.execute(query)
    cursor.fetchall()
    end = time.time()
    return end - begin

def mongo_query(db, col, query):
    begin = time.time()
    db[col].aggregate(query)
    end = time.time()
    return end - begin

def redis_query(graph, query):
    begin = time.time()
    graph.query(query)
    end = time.time()
    return end - begin

In [7]:
def exp(size, insert=True):
    if insert:
        ins(size)
    obs = {}
    # MySQL
    obs[MYSQL] = {}
    try:
        conn = mysql.connector.connect(
            host='localhost',
            database='social',
            user=os.getenv('MYSQL_USER'),
            password=os.getenv('MYSQL_PASS'),
            auth_plugin='mysql_native_password'
        )
        cursor = conn.cursor()
        # (1)
        query = """
            SELECT name, surname 
            FROM user JOIN comment USING (user_id);
        """
        obs[MYSQL][1] = sql_query(query, cursor)
        # (2)
        query = """
            SELECT * FROM user
            JOIN comment USING (user_id)
            WHERE text LIKE "%today%";
        """
        obs[MYSQL][2] = sql_query(query, cursor)
        # (3)
        query = """
            SELECT
                name,
                surname,
                created_at
            FROM user JOIN comment USING (user_id)
            WHERE
                created_at >= CURRENT_DATE AND
                created_at < DATE_ADD(CURRENT_DATE, INTERVAL 1 DAY);
        """
        obs[MYSQL][3] = sql_query(query, cursor)
        conn.close()
    except mysql.connector.Error as error:
        print(f"Failed to connect to MySQL: {error}")
    # MongoDB
    obs[MONGO_DB] = {}
    client = pymongo.MongoClient('mongodb://localhost:27017/')
    db = client['test']
    # (1)
    query = [
        {
            '$lookup': {
                'from': 'comment',
                'localField': '_id',
                'foreignField': 'user_id',
                'as': 'comments',
            },
        },
        {
            '$match': {
                'comments': { '$not': { '$size': 0 } }
            }
        }
    ]
    obs[MONGO_DB][1] = mongo_query(db, 'user', query)
    # (2)
    query = [
        {
            '$match': {
                'text': { '$regex': 'today' }
            },
        },
        {
            '$lookup': {
                'from': 'user',
                'localField': 'user_id',
                'foreignField': '_id',
                'as': 'user',
            }
        }
    ]
    obs[MONGO_DB][2] = mongo_query(db, 'comment', query)
    # (3)
    query = [
        {
            '$match': {
                'created_at': { '$regex': f'^{datetime.date.today()}' }
            },
        },
        {
            '$lookup': {
                'from': 'user',
                'localField': 'user_id',
                'foreignField': '_id',
                'as': 'author',
            }
        }
    ]
    obs[MONGO_DB][3] = mongo_query(db, 'comment', query)
    # Redis
    SOCIAL = 'social'
    obs[REDIS] = {}
    client = redis.Redis(
        host='localhost',
        port=6379,
    )
    graph = Graph(SOCIAL, client)
    # (1)
    query = """
        MATCH r = (u:user)-[:wrote]->(c:comment)
        RETURN r
    """
    obs[REDIS][1] = redis_query(graph, query)
    # (2)
    query = """
        MATCH r = (u:user)-[:wrote]->(c:comment)
        WHERE c.text CONTAINS 'today'
        RETURN r
    """
    obs[REDIS][2] = redis_query(graph, query)
    # (3)
    query = f"""
        MATCH r = (u:user)-[:wrote]->(c:comment)
        WHERE c.created_at STARTS WITH '%s'
        RETURN r
    """ % (f'{datetime.date.today()}')
    obs[REDIS][3] = redis_query(graph, query)
    print(obs)
    return obs

## Experiment

In [17]:
result = {}

SIZES = [
    HUNDRED,
    LAKH
]

for size in SIZES:
    print()
    print(size)
    exp(size)
    result[size] = exp(size, False)


hundred

MySQL
----------
Deleting old records
Inserting users
##########
Inserting comments
##########

MongoDB
----------
Deleting old records
Inserting users
####################
Inserting comments
####################

Redis
----------
Inserting users
##########
Inserting comments
##########
Creating relations
##########
{'MySQL': {1: 0.0039031505584716797, 2: 0.0015053749084472656, 3: 0.0013599395751953125}, 'MongoDB': {1: 0.041106462478637695, 2: 0.0014641284942626953, 3: 0.0008251667022705078}, 'Redis': {1: 0.005754709243774414, 2: 0.0005865097045898438, 3: 0.0005817413330078125}}
{'MySQL': {1: 0.001218557357788086, 2: 0.0004210472106933594, 3: 0.0002951622009277344}, 'MongoDB': {1: 0.009066104888916016, 2: 0.0005085468292236328, 3: 0.0004322528839111328}, 'Redis': {1: 0.0033941268920898438, 2: 0.0003445148468017578, 3: 0.00035762786865234375}}

lakh

MySQL
----------
Deleting old records
Inserting users
##########
Inserting comments
##########

MongoDB
----------
Deleting old 

In [18]:
print(result)

{'hundred': {'MySQL': {1: 0.001218557357788086, 2: 0.0004210472106933594, 3: 0.0002951622009277344}, 'MongoDB': {1: 0.009066104888916016, 2: 0.0005085468292236328, 3: 0.0004322528839111328}, 'Redis': {1: 0.0033941268920898438, 2: 0.0003445148468017578, 3: 0.00035762786865234375}}, 'lakh': {'MySQL': {1: 0.5229933261871338, 2: 0.6053698062896729, 3: 0.0467529296875}, 'MongoDB': {1: 4.8563432693481445, 2: 0.04808664321899414, 3: 0.043874263763427734}, 'Redis': {1: 3.110260486602783, 2: 0.036109209060668945, 3: 0.05787038803100586}}}
